In [1]:
import psycopg2
import pandas as pd
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
#импортируем файл с таблицами и конфиг для подключения
df = pd.read_csv('greenplum_check.csv')
config = pd.read_csv('connections.csv')

In [3]:
df.head(5)

,table_name,complete,base,source,scheme,is_deleted,is_actual,check_date,compare_table,compare_source,raw_count,sql,executed_sql
0,accounting_kpi_period,1,hrm,hrm,public,1,0,0,NaN,NaN,NaN,0,NaN
1,accounting_status,1,hrm,hrm,public,1,0,0,NaN,NaN,NaN,0,NaN
2,application_type,1,hrm,hrm,public,1,0,0,NaN,NaN,NaN,0,NaN
3,billing_domains,1,hrm,hrm,public,1,0,0,NaN,NaN,NaN,0,NaN
4,billing_sections,1,hrm,hrm,public,1,0,0,NaN,NaN,NaN,0,NaN


In [4]:
#основная фукнция обработки селектов
def base_connect(table,base,source,scheme,is_deleted,is_actual,check_date,sql):
    
    # формируем часть запроса по удаленным записям
    if is_deleted == 1 and (base == 'hrm' or base == 'performance_review'):
        deleted_part = " and deleted_at is null"
    elif is_deleted == 1 and base == 'greenplum':
        deleted_part = " and mt_deleted_flag is false "
    else:
        deleted_part = " "
        
    #print('deleted_part')     
    
    # формируем часть запроса по актуальным записям
    if is_actual == 1 and (base == 'hrm' or base == 'performance_review'):
        actual_part = " "
    elif is_actual == 1 and base == 'greenplum':
        actual_part = " and mt_actual_flag "
    else:
        actual_part = " "
        
    #print(actual_part) 
      
    # формируем часть запроса проверки даты  
    if check_date == 1 and (base == 'hrm' or base == 'performance_review'):
        check_date = " "
    elif check_date == 1 and base == 'greenplum':
        check_date = " and end_dttm = '2999-12-31' "
    else:
        check_date = " "
        
    #print(check_date) 
    
    #итоговый запрос
    if  str(sql) == '0':
        select =  "select count(1) from " + scheme + '."' + table + '" where 1=1' + deleted_part + actual_part \
        + check_date
    else:
        select = sql
    
    #print(select)
    
    #подключения
    if base == 'hrm':
        config_info = config[config.name =='hrm']
        connection = psycopg2.connect(dbname=config_info.iloc[0]['base'], user=config_info.iloc[0]['user'],
                            password=config_info.iloc[0]['password'], host=config_info.iloc[0]['host'])
        cursor = connection.cursor()
 
        
    elif base == 'greenplum':
        config_info = config[config.name =='greenplum']
        connection = psycopg2.connect(dbname=config_info.iloc[0]['base'], user=config_info.iloc[0]['user'],
                            password=config_info.iloc[0]['password'], host=config_info.iloc[0]['host'])
        cursor = connection.cursor()
        
            
    elif base == 'performance_review':
        config_info = config[config.name =='performance_review']
        connection = psycopg2.connect(dbname=config_info.iloc[0]['base'], user=config_info.iloc[0]['user'],
                            password=config_info.iloc[0]['password'], host=config_info.iloc[0]['host'])
        cursor = connection.cursor()
        
    elif base == 'jiradb':
        config_info = config[config.name =='jira']
        connection = psycopg2.connect(dbname=config_info.iloc[0]['base'], user=config_info.iloc[0]['user'],
                            password=config_info.iloc[0]['password'], host=config_info.iloc[0]['host'])
        cursor = connection.cursor()
        
    cursor.execute(select)
        
    result = cursor.fetchmany(1)[0][0]
    return(result,select)

In [5]:
#тут укладываем результаты запроса в нужные поля исходного файла
for index, row in df.iterrows():
    if row['complete'] == 1:
        #print(row['table_name'])
        #print(row)
        result,select = base_connect(row['table_name'],row['base'], row['source'], row['scheme'],
                              row['is_deleted'], row['is_actual'],row['check_date'], row['sql'])
        
        df['raw_count'].iloc[index] = result
        df['executed_sql'].iloc[index] = select


In [6]:
df.head(5)

,table_name,complete,base,source,scheme,is_deleted,is_actual,check_date,compare_table,compare_source,raw_count,sql,executed_sql
0,accounting_kpi_period,1,hrm,hrm,public,1,0,0,NaN,NaN,19.0,0,"select count(1) from public.""accounting_kpi_pe..."
1,accounting_status,1,hrm,hrm,public,1,0,0,NaN,NaN,2.0,0,"select count(1) from public.""accounting_status..."
2,application_type,1,hrm,hrm,public,1,0,0,NaN,NaN,9.0,0,"select count(1) from public.""application_type""..."
3,billing_domains,1,hrm,hrm,public,1,0,0,NaN,NaN,56.0,0,"select count(1) from public.""billing_domains"" ..."
4,billing_sections,1,hrm,hrm,public,1,0,0,NaN,NaN,1376.0,0,"select count(1) from public.""billing_sections""..."


In [7]:
#джойним таблицу саму на себя по предустановленным связям
new_df = pd.merge(df, df,  how='left', right_on=['table_name','source'], left_on = ['compare_table','compare_source'])
#добавляем дельту результатов
new_df['delta'] = new_df['raw_count_x'] - new_df['raw_count_y']
#считаем процент ошибок от общего кол-ва записей
new_df['percent'] = round(abs(new_df['delta'])/new_df['raw_count_x']*100,2)

In [8]:
new_df.head(10)

,table_name_x,complete_x,base_x,source_x,scheme_x,is_deleted_x,is_actual_x,check_date_x,compare_table_x,compare_source_x,...,is_deleted_y,is_actual_y,check_date_y,compare_table_y,compare_source_y,raw_count_y,sql_y,executed_sql_y,delta,percent
0,accounting_kpi_period,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,accounting_status,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,application_type,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,billing_domains,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,billing_sections,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,billing_units,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bonus_size_measure,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,bonus_type,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,budget_holder,1,hrm,hrm,public,1,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,category_fired_employee_state_for_hr_bp,1,hrm,hrm,public,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#оставляем нужные поля и только записи, где есть разница результатов
filtered_df = new_df[new_df.delta.notnull() & new_df['delta']!= 0 ].reset_index()
filtered_df_short = filtered_df[['delta','percent','table_name_x','scheme_x','raw_count_x','table_name_y','source_y','raw_count_y']]

In [10]:
#сохраняем ошибки в файл
filtered_csv = filtered_df[['delta','percent','table_name_x','scheme_x','raw_count_x','executed_sql_x','table_name_y','source_y','raw_count_y','executed_sql_y']]
filtered_csv.to_csv('import_errors.csv')

In [11]:
filtered_df_short

,delta,percent,table_name_x,scheme_x,raw_count_x,table_name_y,source_y,raw_count_y
0,-1.0,0.09,budget_holder,raw_hrm_570,1076.0,budget_holder,hrm,1077.0
1,-5.0,0.00,work_profile_tags,raw_hrm_570,113105.0,work_profile_tags,hrm,113110.0
2,26935.0,9.30,accounting,raw_hrm_personal_550,289644.0,accounting,hrm,262709.0
3,-3.0,0.01,demand_dismissal,raw_hrm_personal_550,40862.0,demand_dismissal,hrm,40865.0
4,377.0,0.91,employee,raw_hrm_personal_550,41632.0,employee,hrm,41255.0
5,-2.0,0.04,fired_employee_questionary,raw_hrm_personal_550,5547.0,fired_employee_questionary,hrm,5549.0
6,-2.0,0.04,fired_employee_questionary_appeal,raw_hrm_personal_550,5547.0,fired_employee_questionary_appeal,hrm,5549.0
7,-15.0,0.02,leave,raw_hrm_personal_550,60700.0,leave,hrm,60715.0
8,-12.0,0.03,leave_data,raw_hrm_personal_550,34790.0,leave_data,hrm,34802.0
9,101.0,0.16,salary_history,raw_hrm_personal_550,64556.0,salary_history,hrm,64455.0
